In [32]:
# 3/19/24 Deep Neural Network 16 qubit circuit
# calling qiskit packages, make sure to use qiskit v0.46.0 as that is the only version that supports execute_function
from qiskit import QuantumCircuit, Aer, execute
import json
import numpy as np

# defining all the variables here
Q = 16 # number of qubits
f = 0.1 # defining predetermined frequency value
R = 100 # Predetermined optimal atom replenishing value given by my mentor
S = Q * R # finding number of shots a circuit needs to run in total while assuming a linear relationship between qubits and shots
x = f * S # calculating increment proportional to total number of shots (S)
nc = x # setting unchanging value for the loop
time = 30 # setting the normalization value
filename = "dnn" # setting universal filename to avoid having to set differently
foldername = "DNN 16" # setting universal folder name to avoid having to change it anywhere else
benchmark = "GRAPHINE" # setting universal benchmark name to avoid having to change it everywhere else
bs_var_list = [] # empty list to hold all values of bs_var to see how it changes over time
p_var_list = [] # empty list to hold all vaues of p_var to see how it changes over time


# reading qasm file
qc = QuantumCircuit.from_qasm_file(filename + "_" + str(Q) + ".qasm")

# specifying which backend we will be using
backend = Aer.get_backend("statevector_simulator")

# added variance function instead of having to run it seperately
def variance(shots, b_name, fo_name, f_name):
    # load the output distribution from the json file
    file_path = '/home/aadi/Documents/Neutral-Atom-Technology/Step 2/' + str(b_name) + '/' + str(fo_name) + '/' + str(f_name) + '_counts_' + str(shots) + '_shots.json' 
    with open(file_path, 'r') as file:
        output_distribution = json.load(file)

    # convert the dictionary keys to a list and probabilities to a separate list
    states = list(output_distribution.keys())
    probabilities = list(output_distribution.values())

    # random sampling of basis states based on probability and setting them equal to a list
    sample = np.random.choice(states, size=shots, p=probabilities)

    # random sampling of the probabilites list based on probabilty and setting them equal to a list
    prob_sample = np.random.choice(probabilities, size=shots, p=probabilities)

    # change binary to ints
    int_sample = [int(b, 2) for b in sample]

    # calculating the variance and printing out the results
    sample_var = np.var(int_sample)
    # print("The sample variance is: " + str(sample_var))
    prob_var = np.var(prob_sample)
    # print("The sample variance using probability is : " + str(prob_var))
    
    return sample_var, prob_var


# simple while loop to calculate 
while x <= S:
    
    bs_var = 0 # variance using basis states converted into integers
    p_var = 0 # variance using probability 
    avg_bs_var = 0 # variable to calculate average variance of basis state 
    avg_p_var = 0 # variable to calculate average variance of proabilty
    
    # executing the circuit via backend
    job = execute(qc, backend, shots=x)
    result = job.result()

    # getting statevector results 
    statevector = result.get_statevector()
    # print(statevector)

    # getting counts (output distribution) of the simulation
    counts = result.get_counts(qc)

    # printing shot number to follow along
    # print(str(x))

    # logging the counts to a json file
    with open(filename + '_counts_' + str(int(x)) + '_shots.json', 'w', encoding='utf-8') as f:
        json.dump(counts, f, ensure_ascii=False, indent=4)

    # calculating the variance and normalizing it by running it 30 times and averaging it
    for t in range(1, time+1):
        temp_bs_var, temp_p_var = variance(int(x), benchmark, foldername, filename)
        bs_var += temp_bs_var
        p_var += temp_p_var
        avg_bs_var = bs_var / t
        avg_p_var = p_var / t
        bs_var_list.append(avg_bs_var)  # adding to a list to see how  it changes
        p_var_list.append(avg_p_var)    # adding to a list to see how it changes
    
    
        
    print("\nThe average variance using probability is: " + str(avg_p_var))
    print("The average variance using basis state converted to integers is: " + str(avg_bs_var))
    
    # moving up an increment
    x += nc

/tmp/ipykernel_18523/4163777135.py:66: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(qc, backend, shots=x)



The average variance using probability is: 0.0006609470486864977
The average variance using basis state converted to integers is: 349349697.42826957

The average variance using probability is: 0.0006310370362807278
The average variance using basis state converted to integers is: 360185565.12640977

The average variance using probability is: 0.0006119014744727162
The average variance using basis state converted to integers is: 359961719.5923442

The average variance using probability is: 0.0006476102594976368
The average variance using basis state converted to integers is: 359800097.9249871

The average variance using probability is: 0.0006048783823861697
The average variance using basis state converted to integers is: 361002637.47170174

The average variance using probability is: 0.0006319290962134309
The average variance using basis state converted to integers is: 361779490.73809886

The average variance using probability is: 0.0006162354183537449
The average variance using basis sta

In [12]:
# 3/25/24 Deep Neural Network Statistical Analysis 1600 shots
import numpy as np
import json

# variables defined at the beginning
counts = 1600

# load the output distribution from the json file
file_path = '/home/aadi/Documents/Neutral-Atom-Technology/Step 2/GRAPHINE/DNN 16/dnn_counts_' + str(counts) + '_shots.json' 
with open(file_path, 'r') as file:
    output_distribution = json.load(file)

# convert the dictionary keys to a list and probabilities to a separate list
states = list(output_distribution.keys())
probabilities = list(output_distribution.values())

# random sampling based on probability and setting them equal to a list
sample = np.random.choice(states, size=counts, p=probabilities)

prob_sample = np.random.choice(probabilities, size=counts, p=probabilities)

# change binary to ints
int_sample = [int(b, 2) for b in sample]


# calculating the variance and printing out the results
sample_var = np.var(int_sample)
print("The sample variance is: " + str(sample_var))
prob_var = np.var(prob_sample)
print("The sample variance using probability is : " + str(prob_var))


The sample variance is: 372046197.84436095
The sample variance using probability is : 0.0006575078253964147
